In [1]:

import pandas as pd
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols
import tweepy
import json
import os
from collections import Counter
import numpy as np
from collections import defaultdict
from matplotlib import pyplot as plt
import time
from datetime import datetime
from dateutil.parser import parse

In [2]:
# load tweets.json file
def load_json(file_name):
    result=[]
    with open(file_name, 'r') as fp:
        line=fp.readline()
        while line:
            tjson=json.loads(line) #decode json
            result.append({
                 "Date":tjson["created_at"],
                "id":tjson["id_str"],
                 "user_name":tjson["user"]["name"],
                   
                    "favorite_count":tjson["favorite_count"], #number of favorites
                    "retweet_count":tjson["retweet_count"], #get number of retweets
                    "user_id_str":tjson["user"]["id_str"], #get user id_str
                
                
            })

            line=fp.readline()
    return result

# load tweets.json file
def load_old_json(file_name):
    result=[]
    with open(file_name, 'r') as fp:
        line=fp.readline()
        while line:
            tjson=json.loads(line) #decode json
            result.append({
                "Date":tjson["Date"],
                "id":tjson["id"],
                "favorite_count":tjson["favorite_count"], #number of favorites
                "retweet_count":tjson["retweet_count"], #number of retweets
                "text":tjson["text"]
            })
            line=fp.readline()
    return result


In [4]:
data_list=load_json("tweets.json")
# data_list=load_old_json("old_tweets.json")
df=pd.DataFrame(data=data_list)
print('done')
#df = df.drop_duplicates(subset=['id'], keep=False)

#converting date into more readable form 
Date_list= df["Date"].values
dates = []
for date in Date_list:
    dates.append(parse(date).date())
df["Date"]=dates
print(df)

FileNotFoundError: [Errno 2] No such file or directory: 'tweets.json'

In [ ]:
### Sentiment Analysis using an API sentimentIntensityAnalyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

SIA = SentimentIntensityAnalyzer()
#list of sentiments
sentiments = []
#on the basis of each texts in the dataframe 
for t in df["text"].values:
    sentiment_dict = SIA.polarity_scores(t)
    
    # decide sentiment as positive, negative and neutral 
    if sentiment_dict['compound'] >= 0.05 : 
        sentiments.append("Positive") 
        
    elif sentiment_dict['compound'] <= - 0.05 : 
        sentiments.append("Negative") 
    else :
        sentiments.append("Neutral")
  #datafram column for sentiment using sentiment scores      
df["sentiment"]= sentiments

#functions to return the list of positive, negative and neutral sentiments
def count_pos(series):
    return len([x for x in series if x=="Positive"])
def count_neg(series):
    return len([x for x in series if x=="Negative"])
def count_nue(series):
    return len([x for x in series if x=="Neutral"])
#making a csv file of sentiment data

df.to_csv("data_with_sent.csv", index=True)


In [ ]:

#groupby date
group=df.groupby(by=["Date"])
#found aggregate of tweet id, favorite count, retweet coint and sentiment 
new_df=group.aggregate({
    "id":["count"],
    "favorite_count":["sum"],
    "retweet_count":["sum"],
    "sentiment":[count_pos, count_neg, count_nue]
})
#rename id_str to tweet_count 
new_df.rename(columns={"id_str":"tweet_count"}, inplace=True)


print(new_df)
#new csv file for old_climate data
new_df.to_csv("old_climate_data.csv", index=True)
